In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
import csv
from sklearn.linear_model import LinearRegression
#linreg gives straight line
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
#doesnt work idk
from sklearn.ensemble import RandomForestClassifier
#doesnt work
from sklearn.ensemble import RandomForestRegressor
#gives me 1 entire value
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor
#gives me 1 value
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import MultiTaskLasso
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
import time
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from geopy import distance
import preprocess as pre

In [2]:
train = pd.read_csv("ais_train.csv", sep='|')
test = pd.read_csv("ais_test.csv")
train['time'] = pd.to_datetime(train['time'])
test['time'] = pd.to_datetime(test['time'])

train.tail()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
1522060,2024-05-07 23:59:07,359.1,13.4,0,1,0,05-08 05:00,52.19131,-5.82223,clh6aqawa0002gh0zypfa5dut,634c4de270937fc01c3a7417
1522061,2024-05-07 23:59:08,12.3,17.1,0,13,0,05-10 03:00,38.96142,-12.00502,61e9f3aeb937134a3c4bfe43,634c4de270937fc01c3a76a1
1522062,2024-05-07 23:59:08,269.8,14.9,-1,270,0,05-15 23:00,49.71372,-5.22042,61e9f43db937134a3c4c0169,634c4de270937fc01c3a787b
1522063,2024-05-07 23:59:08,8.0,18.7,0,6,0,05-08 12:45,38.27895,10.78280,61e9f469b937134a3c4c029b,61d3781293c6feb83e5eb73b
1522064,2024-05-07 23:59:08,336.0,14.3,5,337,0,05-07 23:00,38.98635,-75.13275,62080cff66fc0a8e43c6123a,61d38528b7b7526e1adf3e6f


In [3]:
start = time.time()
#pre.ais_trainCleanup("~/proj2/ML/Henrik/ais_train.csv","ais_clean.csv")
print(time.time()-start)

870.0213673114777


In [3]:
cleandata = pd.read_csv("ais_clean.csv",sep="|")
cleandata['time'] = pd.to_datetime(cleandata['time'])
cleandata.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,...,time_at_sea,dist_s_l,tot_dist,time_s_l,speed,delta_lat,delta_lon,delta_time,last_lon,last_lat
0,2024-01-01 00:17:17,73.9,0.1,12,120,1,2024-12-28 05:00:00,-26.77586,153.23453,61e9f3acb937134a3c4bfe23,...,0 days 00:06:01,0.033917,0.033917,361,0.338229,0.00026,0.00018,361.0,153.23435,-26.77612
1,2024-01-01 00:18:57,150.7,0.1,0,251,0,2024-12-31 04:00:00,38.47387,15.91592,61e9f3cab937134a3c4bff01,...,0 days 00:02:39,0.002618,0.002618,159,0.059274,0.00000,-0.00003,159.0,15.91595,38.47387
2,2024-01-01 00:19:39,56.4,12.9,-16,56,0,2024-12-30 20:00:00,33.90815,130.92404,61e9f3e1b937134a3c4bff59,...,0 days 00:05:34,2.085388,2.085388,334,22.477237,0.00412,0.02200,334.0,130.90204,33.90403
3,2024-01-01 00:20:35,114.5,0.0,0,275,5,2024-01-03 13:30:00,43.44237,-3.82309,clh6aqawa0002gh0zypfa5dut,...,0 days 00:12:00,0.005775,0.005775,720,0.028873,-0.00001,0.00007,720.0,-3.82316,43.44238
4,2024-01-01 00:21:37,210.4,0.0,-5,122,1,2024-12-31 04:00:00,40.68658,29.31613,61e9f3c7b937134a3c4bfedf,...,0 days 00:06:00,0.025431,0.025431,360,0.254307,-0.00021,0.00012,360.0,29.31601,40.68679


In [4]:
#endrer tid og 1hoter vessel
warnings.filterwarnings('ignore')

def preprocess(df):
    data = df.copy()
    time3 = []
    for i in data['time']:
        time3.append(int(i.timestamp()-1704067200))
    data['time'] = np.array(time3, dtype = 'float32')
    data = pd.get_dummies(data, columns=['vesselId'], drop_first=True)
    return(data)

def timefix(df):
    data = df.copy()
    time3 = []
    for i in data['time']:
        time3.append(int(i.timestamp()-1704067200))
    data['time'] = np.array(time3, dtype = 'float32')
    return data

def speed(x,time_a,time_b):
    if time_a == time_b:
        return 0
    else:
        return min((x)/(time_b-time_a),50)
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df
#Må brukes på enkeltbåt
def deltafier(df):
    data = df.copy()
    data["delta_lat"] = data["latitude"]-data["latitude"].shift(1)
    data["delta_lon"] = (data["longitude"]-data["longitude"].shift(1)+180)%360-180
    data["delta_time"] = timefix(data)["time"]-timefix(data)["time"].shift(1)
    data["last_lon"] = data["longitude"].shift(1)
    data["last_lat"] = data["latitude"].shift(1)
    return data[1:]

In [6]:
start = time.time()
trainingdata5 = cleandata[["vesselId","delta_time","time","last_lat","last_lon"]]
goaldata5 = cleandata[["delta_lat","delta_lon"]]
trainingdata5 = preprocess(trainingdata5)
trainingdata5 = trainingdata5.drop(columns = "time")
#trainingdata5 = pd.get_dummies(trainingdata5, columns=['navstat'], drop_first=True)
trainingdata5 = trainingdata5.reindex(sorted(trainingdata5.columns), axis=1)
print(time.time()-start)

5.607004165649414


In [49]:
start = time.time()
model5 = xgb.XGBRegressor()
model5.fit(trainingdata5,goaldata5)
print(time.time()-start)

164.21562433242798


In [8]:
uniquevessel = [i for i in test.copy()['vesselId'].unique()]
traincopy = train.copy()
bettertraining = traincopy[0:1]
for i in uniquevessel:
    boat = traincopy[traincopy['vesselId'] == i]
    bettertraining.loc[len(bettertraining.index)] = boat.iloc[-1]
bettertraining = bettertraining[1:]
bettertraining.tail()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
211,2024-04-26 13:50:08,188.2,17.1,2,187,0,04-27 20:00,36.66386,125.90680,631e3eb0192150e13fa41628,61d37a531366c3998241d97d
212,2024-04-30 18:39:21,251.9,14.3,9,253,0,05-03 09:30,34.64546,139.47670,61e9f41cb937134a3c4c0091,634c4de270937fc01c3a7132
213,2024-04-30 07:08:07,137.5,4.5,-15,137,0,04-29 14:00,32.79268,-79.92353,61e9f39fb937134a3c4bfdcd,61d3847cb7b7526e1adf3d1b
214,2024-04-21 08:15:03,6.2,16.1,0,5,0,04-19 03:30,25.29443,121.26813,6326eed6c46d6a20d22ca319,634c4de270937fc01c3a7744
215,2024-05-03 22:32:02,76.9,13.8,0,79,0,05-11 22:00,1.27494,104.27201,61e9f45ab937134a3c4c021b,61d37a571366c3998241d983


In [43]:
start = time.time()
test5 = test.copy()
uniqueVesselId = test5["vesselId"].unique()
test5_processed_list = []
for id in uniqueVesselId:
    boat = test5[test5["vesselId"] == id].reset_index(drop=True)
    boat["delta_time"] = timefix(boat)["time"]-timefix(boat)["time"].shift(1)
    boat.at[0,"delta_time"] = timefix(boat).at[0,"time"]-timefix(bettertraining[bettertraining["vesselId"]==id].reset_index(drop=True)).at[0,"time"]
    boat.at[0,"last_lat"] = bettertraining[bettertraining["vesselId"]==id].reset_index(drop=True).at[0,"latitude"]
    boat.at[0,"last_lon"] = bettertraining[bettertraining["vesselId"]==id].reset_index(drop=True).at[0,"longitude"]
        #Appending rows to list
    for _, row in boat.iterrows():
        test5_processed_list.append(row.to_dict())

test5_processed = pd.DataFrame(test5_processed_list).sort_values(by="vesselId").reset_index(drop=True)
test5_processed["index"] = test.sort_values(by = ["vesselId","time"]).reset_index()["index"]
test5_processed = test5_processed.sort_values(by = "index").drop(columns="index").reset_index(drop=True)

print(time.time()-start)

5.701102256774902


In [44]:
test5p = test5_processed.copy()
#test5p = pd.merge(test5p,bettertraining.copy(), on = "vesselId",how = "left")
#test5p['time']=test5p['time_x']
test5p = test5p[["last_lat","last_lon","vesselId","time","delta_time"]]
test5p = test5p.sort_values(by=["vesselId","time"])
test5p = preprocess(test5p)

#test5p = pd.get_dummies(test5p, columns=['navstat'], drop_first=True)

for column in trainingdata5:
    if column not in test5p:
        test5p[column] = False


test5p = test5p.reindex(sorted(test5p.columns), axis=1).reset_index()


test5p.tail()

,index,delta_time,last_lat,last_lon,time,vesselId_61e9f38eb937134a3c4bfd8d,vesselId_61e9f38eb937134a3c4bfd8f,vesselId_61e9f38eb937134a3c4bfd91,vesselId_61e9f390b937134a3c4bfd93,vesselId_61e9f391b937134a3c4bfd95,...,vesselId_clh682fbq000qghxd3vn5xo01,vesselId_clh682fbq000rghxd2u75zdbn,vesselId_clh6aqawa0000gh0z7nc8e6nd,vesselId_clh6aqawa0001gh0zmijpuho1,vesselId_clh6aqawa0002gh0zypfa5dut,vesselId_clh6aqawa0003gh0zu0aznvt2,vesselId_clh6aqawa0004gh0z12aogec9,vesselId_clh6aqawa0005gh0z64y4xyip,vesselId_clh6aqawa0006gh0zje911dl3,vesselId_clh6aqawa0007gh0z9h6zi9bo
51734,5118,1213.0,NaN,NaN,11486253.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
51735,4953,1232.0,NaN,NaN,11487485.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
51736,4845,1215.0,NaN,NaN,11488700.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
51737,4728,1261.0,NaN,NaN,11489961.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
51738,4599,1182.0,NaN,NaN,11491143.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [55]:
#lat,lon
start = time.time()
worker = test5p.copy().drop(columns=["index"])
worker["vesselId"] = test5_processed.sort_values(by = ["vesselId","time"]).reset_index()["vesselId"]

def predi(data,vid,tim):
    work = data.copy().reset_index(drop=True)
    for i in range(len(work)-1):
        pred = model5.predict(work[i:i+1])[0]
        work.at[i+1,"last_lat"]=work.at[i,"last_lat"]+pred[0]
        work.at[i+1,"last_lon"]=(work.at[i,"last_lon"]+pred[1]+180)%360-180    
    last = model5.predict(work[-1:])[0]
    work["latitude"]=work["last_lat"].shift(-1)
    work["longitude"]=work["last_lon"].shift(-1)
    work.at[len(work)-1,"latitude"]=work.at[len(work)-1,"last_lat"]+last[0]
    work.at[len(work)-1,"longitude"]=(work.at[len(work)-1,"last_lon"]+last[1]+180)%360-180
    work["vesselId"]=id
    work["time"]=tim
    return work

uniqueVessel = worker["vesselId"].unique()
dog_list = []
for id in uniqueVesselId:
    #Isolate a single boat and clean
    boat = worker[worker["vesselId"] == id].drop(columns=["vesselId"])
    tim = boat["time"]
    boat = predi(boat.drop(columns="time"),id,tim)
    for _, row in boat.iterrows():
        dog_list.append(row.to_dict())

worker = pd.DataFrame(dog_list).sort_values(by=["vesselId","time"]).reset_index(drop=True)

worker["index"]=test5p["index"]
worker = worker.sort_values(by=["index"]).reset_index(drop=True)

print(time.time()-start)
worker.tail()

KeyboardInterrupt: 

In [59]:
pd.DataFrame(dog_list).sort_values(by=["vesselId","time"]).reset_index(drop=True).tail()

,delta_time,last_lat,last_lon,vesselId_61e9f38eb937134a3c4bfd8d,vesselId_61e9f38eb937134a3c4bfd8f,vesselId_61e9f38eb937134a3c4bfd91,vesselId_61e9f390b937134a3c4bfd93,vesselId_61e9f391b937134a3c4bfd95,vesselId_61e9f391b937134a3c4bfd97,vesselId_61e9f392b937134a3c4bfd99,...,vesselId_clh6aqawa0002gh0zypfa5dut,vesselId_clh6aqawa0003gh0zu0aznvt2,vesselId_clh6aqawa0004gh0z12aogec9,vesselId_clh6aqawa0005gh0z64y4xyip,vesselId_clh6aqawa0006gh0zje911dl3,vesselId_clh6aqawa0007gh0z9h6zi9bo,latitude,longitude,vesselId,time
24151,1270.0,51.093568,3.545937,False,False,False,False,False,False,False,...,False,False,False,False,True,False,51.094080,3.544977,clh6aqawa0006gh0zje911dl3,NaN
24152,1296.0,51.094080,3.544977,False,False,False,False,False,False,False,...,False,False,False,False,True,False,51.094591,3.544016,clh6aqawa0006gh0zje911dl3,NaN
24153,1219.0,51.094591,3.544016,False,False,False,False,False,False,False,...,False,False,False,False,True,False,51.095103,3.543055,clh6aqawa0006gh0zje911dl3,NaN
24154,1228.0,51.095103,3.543055,False,False,False,False,False,False,False,...,False,False,False,False,True,False,51.095615,3.542094,clh6aqawa0006gh0zje911dl3,NaN
24155,124.0,51.095615,3.542094,False,False,False,False,False,False,False,...,False,False,False,False,True,False,51.096127,3.541134,clh6aqawa0006gh0zje911dl3,NaN


In [57]:
ramg = [i for i in range(51739)]
#res2 = [ramg,[(i[1]+180)%360-180 for i in res],[(i[0]+90)%180-90 for i in res]]
#res3 = [ramg,[i[0] for i in res2],[i[0] for i in res1]]
res3 = [ramg,worker["longitude"],worker["latitude"]]
with open('res.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(["ID","longitude_predicted","latitude_predicted"])
    write.writerows(zip(*res3))
df = pd.read_csv('res.csv')
df.to_csv('res.csv', index=False)

KeyError: 'longitude'